# Question 1

In [1]:
!bentoml --version

bentoml, version 1.0.7.post43+g7a1df086


### bentoml, version 1.0.7.post43+g7a1df086

# Question 2

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb

### Data preparation

In [4]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-06-trees/CreditScoring.csv'
df = pd.read_csv(data)

In [5]:
df.columns = df.columns.str.lower()

status_values = {
    1: 'ok',
    2: 'default',
    0: 'unk'
}

df.status = df.status.map(status_values)

home_values = {
    1: 'rent',
    2: 'owner',
    3: 'private',
    4: 'ignore',
    5: 'parents',
    6: 'other',
    0: 'unk'
}

df.home = df.home.map(home_values)

marital_values = {
    1: 'single',
    2: 'married',
    3: 'widow',
    4: 'separated',
    5: 'divorced',
    0: 'unk'
}

df.marital = df.marital.map(marital_values)

records_values = {
    1: 'no',
    2: 'yes',
    0: 'unk'
}

df.records = df.records.map(records_values)

job_values = {
    1: 'fixed',
    2: 'partime',
    3: 'freelance',
    4: 'others',
    0: 'unk'
}

df.job = df.job.map(job_values)

for c in ['income', 'assets', 'debt']:
    df[c] = df[c].replace(to_replace=99999999, value=np.nan)

df = df[df.status != 'unk'].reset_index(drop=True)

In [6]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=11)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = (df_train.status == 'default').astype('int').values
y_test = (df_test.status == 'default').astype('int').values

del df_train['status']
del df_test['status']

In [7]:
dv = DictVectorizer(sparse=False)

train_dicts = df_train.fillna(0).to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

test_dicts = df_test.fillna(0).to_dict(orient='records')
X_test = dv.transform(test_dicts)

### Random forest

In [8]:
rf = RandomForestClassifier(n_estimators=200,
                            max_depth=10,
                            min_samples_leaf=3,
                            random_state=1)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=3, n_estimators=200,
                       random_state=1)

### XGBoost

In [10]:
dtrain = xgb.DMatrix(X_train, label=y_train)

In [11]:
xgb_params = {
    'eta': 0.1, 
    'max_depth': 3,
    'min_child_weight': 1,

    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=175)

### BentoML

In [12]:
import bentoml

In [13]:
bentoml.xgboost.save_model(
    'credit_risk_model',
    model,
    custom_objects={
        'dictVectorizer': dv
    })

Model(tag="credit_risk_model:vpmqlxss6gwcaagy", path="C:\Users\Kartik Powar\bentoml\models\credit_risk_model\vpmqlxss6gwcaagy\")

In [14]:
import json

In [15]:
request = df_test.iloc[0].to_dict()
print(json.dumps(request, indent=2))

{
  "seniority": 3,
  "home": "owner",
  "time": 36,
  "age": 26,
  "marital": "single",
  "records": "no",
  "job": "freelance",
  "expenses": 35,
  "income": 0.0,
  "assets": 60000.0,
  "debt": 3000.0,
  "amount": 800,
  "price": 1000
}


### Answer: 197.79 KiB

# Question 3

### Answer

In [19]:
from pydantic import BaseModel

In [18]:
class UserProfile(BaseModel):
    name: str
    age: int
    country: str
    rating: float

# Question 4

In [22]:
import bentoml
bentoml.models.get("mlzoomcamp_homework:qtzdz3slg6mwwdu5")

Model(tag="mlzoomcamp_homework:qtzdz3slg6mwwdu5", path="C:\Users\Kartik Powar\bentoml\models\mlzoomcamp_homework\qtzdz3slg6mwwdu5")

In [23]:
!bentoml models get mlzoomcamp_homework:qtzdz3slg6mwwdu5

name: mlzoomcamp_homework                                                      
version: qtzdz3slg6mwwdu5                                                      
module: bentoml.sklearn                                                        
labels: {}                                                                     
options: {}                                                                    
metadata: {}                                                                   
context:                                                                       
  framework_name: sklearn                                                      
  framework_versions:                                                          
    scikit-learn: 1.1.1                                                        
  bentoml_version: 1.0.7                                                       
  python_version: 3.9.12                                                       
signatures:                             

 ### Answer: 1.1.1

# Question 5

In [25]:
model_ref = bentoml.sklearn.get("mlzoomcamp_homework:qtzdz3slg6mwwdu5")

In [26]:
model_ref.custom_objects

{}

In [35]:
!curl -X POST -F 'input=[[6.4,3.5,4.5,1.2]]' http://localhost:3000/classify

"An error has occurred in BentoML user code when handling this request, find the error details in server logs"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   271  100   110  100   161    249    365 --:--:-- --:--:-- --:--:--   615


### Answer: 1

# Question 6

In [32]:
!curl -O https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel2.bentomodel

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1728  100  1728    0     0   1401      0  0:00:01  0:00:01 --:--:--  1402
100  1728  100  1728    0     0   1401      0  0:00:01  0:00:01 --:--:--  1401


In [33]:
!bentoml models import coolmodel2.bentomodel

Model(tag="mlzoomcamp_homework:jsi67fslz6txydu5") imported.


In [34]:
!bentoml models get mlzoomcamp_homework:jsi67fslz6txydu5

['name: mlzoomcamp_homework                                                      ',
 'version: jsi67fslz6txydu5                                                      ',
 'module: bentoml.sklearn                                                        ',
 'labels: {}                                                                     ',
 'options: {}                                                                    ',
 'metadata: {}                                                                   ',
 'context:                                                                       ',
 '  framework_name: sklearn                                                      ',
 '  framework_versions:                                                          ',
 '    scikit-learn: 1.1.1                                                        ',
 '  bentoml_version: 1.0.7                                                       ',
 '  python_version: 3.9.12                                                  

### Answer: model 2